In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import load_model
from keras_tqdm import TQDMNotebookCallback
from tqdm import tqdm_notebook

from fastmri_recon.data.fastmri_sequences import ZeroFilled2DSequence
from fastmri_recon.helpers.image_tboard_cback import TensorBoardImage
from fastmri_recon.models.unet import unet

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [3]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [4]:
# generators
AF = 4
train_gen = ZeroFilled2DSequence(train_path, af=AF, norm=True)
val_gen = ZeroFilled2DSequence(val_path, af=AF, norm=True)

In [5]:
run_params = {
    'n_layers': 4, 
    'pool': 'max', 
    "layers_n_channels": [16, 32, 64, 128], 
    'layers_n_non_lins': 2,
#     'n_layers': 2,
}
n_epochs = 1
run_id = f'unet_wo_lastrelu_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_wo_lastrelu_af4_1572013433


In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=500)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
selected_slice = 15
tboard_image_cback = TensorBoardImage(
    log_dir=log_dir + '/images',
    image=train_gen[0][1][selected_slice:selected_slice+1],
    # NOTE: for cross-domain slice has to be on kspace and mask
    model_input=train_gen[0][0][selected_slice:selected_slice+1],
)

In [7]:
model = unet(input_size=(320, 320, 1), lr=1e-3, **run_params)
print(model.summary())

W1025 16:23:56.657170 139849088984832 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 320, 320, 16) 160         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 320, 320, 16) 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 160, 160, 16) 0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [8]:
# model.fit_generator(
#     train_gen, 
#     steps_per_epoch=n_volumes_train, 
#     epochs=n_epochs,
#     validation_data=val_gen,
#     validation_steps=n_volumes_val,
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, lr_on_plat_cback],
#     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
# )

In [9]:
# overfitting trials

data = train_gen[0]
val_data = val_gen[0]
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0].shape[0], 
    callbacks=[tboard_cback, chkpt_cback, tqdm_cb, tboard_image_cback],
    epochs=500, 
    verbose=0, 
    shuffle=False,
)

W1025 16:24:02.787692 139849088984832 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W1025 16:24:02.797880 139849088984832 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1025 16:24:02.799185 139849088984832 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks/tensorboard_v1.py:203: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))
W1025 16:24:03.263882 139849088984832 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/image_tboard_cback.py:26: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.

